In [37]:
from sagas.ofbiz.connector import OfbizConnector
from sagas.ofbiz.finder import Finder
oc=OfbizConnector()
finder=Finder(oc)
print(oc.delegator.getDelegatorName())

default


In [38]:
def clear_all():
    oc.delegator.removeAll("TestingNodeMember") 
    oc.delegator.removeAll("TestingNode")   
    oc.delegator.removeAll("Testing")  
    oc.delegator.removeAll("TestingType") 

clear_all()

In [39]:
def prepare():
    create=finder.create
    UtilDateTime=finder.oc.j.UtilDateTime
    nowTimestamp=finder.now()
    
    create("TestingType", "testingTypeId", "PERFOMFINDTEST")

    create("Testing", "testingId", "PERF_TEST_1", "testingTypeId", "PERFOMFINDTEST", "testingName", "nice name one")
    create("Testing", "testingId", "PERF_TEST_2", "testingTypeId", "PERFOMFINDTEST", "testingName", "nice other name two")
    create("Testing", "testingId", "PERF_TEST_3", "testingTypeId", "PERFOMFINDTEST", "testingName", "medium name three")
    create("Testing", "testingId", "PERF_TEST_4", "testingTypeId", "PERFOMFINDTEST", "testingName", "bad nme four")
    create("Testing", "testingId", "PERF_TEST_5", "testingTypeId", "PERFOMFINDTEST", "testingName", "nice name one")
    create("Testing", "testingId", "PERF_TEST_6", "testingTypeId", "PERFOMFINDTEST")
    create("Testing", "testingId", "PERF_TEST_7", "testingTypeId", "PERFOMFINDTEST")
    create("Testing", "testingId", "PERF_TEST_8", "testingTypeId", "PERFOMFINDTEST")
    create("Testing", "testingId", "PERF_TEST_9", "testingTypeId", "PERFOMFINDTEST")

    create("TestingNode", "testingNodeId", "NODE_1", "description", "Date Node")
    create("TestingNodeMember", "testingNodeId", "NODE_1", "testingId", "PERF_TEST_5",
            "fromDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, 1),
            "thruDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, 3),
            "extendFromDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, -1),
            "extendThruDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, 3))
    create("TestingNodeMember", "testingNodeId", "NODE_1", "testingId", "PERF_TEST_6",
            "fromDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, -1),
            "thruDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, 1),
            "extendFromDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, -1),
            "extendThruDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, 3))
    create("TestingNodeMember", "testingNodeId", "NODE_1", "testingId", "PERF_TEST_7",
            "fromDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, -1),
            "thruDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, 1),
            "extendFromDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, -1),
            "extendThruDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, 3))
    create("TestingNodeMember", "testingNodeId", "NODE_1", "testingId", "PERF_TEST_8",
            "fromDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, -3),
            "thruDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, 1),
            "extendFromDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, -1),
            "extendThruDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, 3))
    create("TestingNodeMember", "testingNodeId", "NODE_1", "testingId", "PERF_TEST_9",
            "fromDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, -3),
            "thruDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, -1),
            "extendFromDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, -1),
            "extendThruDate", UtilDateTime.addDaysToTimestamp(nowTimestamp, -3))

prepare()

In [41]:
def tests_op(val, op):
    print('testing', op, val)
    inputs=finder.hash_map("testingName", val, "testingName_op", op)
    ok, result=finder.find("Testing", inputs)
    if ok:
        for rec in result:
            print("\t", rec['testingName'])
    else:
        print("\t", 'error', result)

tests_op('name', 'contains')
tests_op('nice', 'like')
tests_op('name', 'not-like')
tests_op('name', 'not-contains')

testing contains name
	 nice name one
	 nice other name two
	 medium name three
	 nice name one
testing like nice
	 nice name one
	 nice other name two
	 nice name one
testing not-like name
	 nice name one
	 nice other name two
	 medium name three
	 bad nme four
	 nice name one
testing not-contains name
	 bad nme four


In [42]:
def filter_by_date(filter=True):
    UtilDateTime=finder.oc.j.UtilDateTime
    # nowTimestamp=finder.now()
    userLogin=finder.user
    if filter:
        filter_val="Y"
    else:
        filter_val="N"

    inputFields=finder.hash_map("testingNodeId", "NODE_1")
    performFindMap=finder.hash_map("userLogin", userLogin, 
                               "entityName", "TestingNodeMember", 
                               "inputFields", inputFields, 
                               "filterByDate", filter_val, 
                               "filterByDateValue", UtilDateTime.nowTimestamp())

    result = oc.dispatcher.runSync("performFind", performFindMap)
    print(finder.success(result))
    listIt = result['listIt']
    foundElements = listIt.getCompleteList()
    for e in foundElements:
        print(e['testingId'], e['fromDate'])

filter_by_date(True)

True
PERF_TEST_6 2019-01-21 23:10:28.633
PERF_TEST_7 2019-01-21 23:10:28.633
PERF_TEST_8 2019-01-19 23:10:28.633


In [43]:
def desc_relations(entity_name):
    ent=oc.delegator.getModelEntity(entity_name)
    print(ent.getPlainTableName(), ent.getTitle(), ent.getDescription())
    for rel in ent.getRelationsList(True, True, True):
        print("\t", rel.getType(), rel)

desc_relations("Testing")
desc_relations("TestingNode")

TESTING Testing None
	 one Testing->TestingType[testingTypeId:testingTypeId]
	 many Testing->TestingItem[testingId:testingId]
	 many Testing->TestingNodeMember[testingId:testingId]
TESTING_NODE Testing Node None
	 one TestingNode->PrimaryParentTestingNode[primaryParentNodeId:testingNodeId]
	 many TestingNode->PrimaryChildTestingNode[testingNodeId:primaryParentNodeId]
	 many TestingNode->TestingNodeMember[testingNodeId:testingNodeId]


In [44]:
def find_one(entity, params):
    return oc.delegator.findOne(entity, params, True)
def jlist(*args):
    m = oc.j.ArrayList()
    for e in args:
        m.append(e)
    return m

ent=find_one("Testing", finder.hash_map("testingId", "PERF_TEST_1"))
rel=ent.getRelatedOne("TestingType", False)
print(rel)
items=ent.getRelated("TestingNodeMember", jlist('testingId'))
print(len(items))
for item in items:
    print(item)

{'description': None, 'lastUpdatedStamp': JavaObject id=o173, 'lastUpdatedTxStamp': JavaObject id=o174, 'createdTxStamp': JavaObject id=o175, 'testingTypeId': 'PERFOMFINDTEST', 'createdStamp': JavaObject id=o176}
0


In [45]:
ent=find_one("TestingNode", finder.hash_map("testingNodeId", "NODE_1"))
items=ent.getRelated("TestingNodeMember", jlist('testingNodeId'))
print("members", len(items))
for item in items:
    print(item['testingId'], item)

members 5
PERF_TEST_5 {'fromDate': JavaObject id=o190, 'lastUpdatedStamp': JavaObject id=o191, 'extendFromDate': JavaObject id=o192, 'createdTxStamp': JavaObject id=o193, 'createdStamp': JavaObject id=o194, 'testingId': 'PERF_TEST_5', 'lastUpdatedTxStamp': JavaObject id=o195, 'extendThruDate': JavaObject id=o196, 'testingNodeId': 'NODE_1', 'thruDate': JavaObject id=o197}
PERF_TEST_6 {'fromDate': JavaObject id=o202, 'lastUpdatedStamp': JavaObject id=o203, 'extendFromDate': JavaObject id=o204, 'createdTxStamp': JavaObject id=o205, 'createdStamp': JavaObject id=o206, 'testingId': 'PERF_TEST_6', 'lastUpdatedTxStamp': JavaObject id=o207, 'extendThruDate': JavaObject id=o208, 'testingNodeId': 'NODE_1', 'thruDate': JavaObject id=o209}
PERF_TEST_7 {'fromDate': JavaObject id=o214, 'lastUpdatedStamp': JavaObject id=o215, 'extendFromDate': JavaObject id=o216, 'createdTxStamp': JavaObject id=o217, 'createdStamp': JavaObject id=o218, 'testingId': 'PERF_TEST_7', 'lastUpdatedTxStamp': JavaObject id=o